# Fiscal Policy Impact Calculator - Example Usage

This notebook demonstrates how to use the fiscal policy scoring engine programmatically.

**Features covered:**
- Basic policy scoring
- Policy types (TCJA, corporate, credits, estate, payroll, AMT)
- Distributional analysis
- Dynamic scoring with FRB/US-calibrated multipliers
- Policy packages

In [ ]:
# Setup - add parent directory to path for imports
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd

# Core imports
from fiscal_model import FiscalPolicyScorer, TaxPolicy, CapitalGainsPolicy, SpendingPolicy
from fiscal_model.baseline import CBOBaseline

## 1. Basic Policy Scoring

The simplest use case: score a tax policy with a rate change affecting incomes above a threshold.

In [ ]:
# Initialize the scorer
scorer = FiscalPolicyScorer(use_real_data=True)

# Define a simple tax increase on high earners
policy = TaxPolicy(
    name="High-Income Tax Increase",
    rate_change=0.026,  # +2.6 percentage points
    affected_income_threshold=400_000,  # Incomes above $400K
    taxable_income_elasticity=0.25,  # Standard ETI (Saez et al. 2012)
    duration_years=10
)

# Score the policy
result = scorer.score_policy(policy, dynamic=False)

print(f"Policy: {policy.name}")
print(f"Static Revenue Effect: ${result.static_revenue_effect:,.0f}B")
print(f"Behavioral Offset: ${result.behavioral_offset:,.0f}B")
print(f"Final 10-Year Impact: ${result.final_deficit_effect:,.0f}B")

## 2. TCJA Extension Scoring

The Tax Cuts and Jobs Act (TCJA) provisions expire in 2025. The model scores full extension at ~$4.6T (CBO estimate: $4.6T).

In [ ]:
from fiscal_model.tcja import create_tcja_extension

# Full TCJA extension (all provisions)
tcja_full = create_tcja_extension(extend_all=True, keep_salt_cap=True)
result_tcja = scorer.score_policy(tcja_full, dynamic=False)

print("TCJA Full Extension")
print(f"10-Year Cost: ${result_tcja.final_deficit_effect:,.0f}B")
print(f"CBO Official: $4,600B")
print(f"Error: {abs(result_tcja.final_deficit_effect - 4600) / 4600 * 100:.1f}%")

# Show component breakdown
if hasattr(tcja_full, 'get_component_scores'):
    print("\nComponent Breakdown:")
    for component, score in tcja_full.get_component_scores().items():
        print(f"  {component}: ${score:,.0f}B")

In [ ]:
# TCJA without SALT cap (more expensive)
tcja_no_salt = create_tcja_extension(extend_all=True, keep_salt_cap=False)
result_no_salt = scorer.score_policy(tcja_no_salt, dynamic=False)

print("TCJA Extension WITHOUT SALT Cap")
print(f"10-Year Cost: ${result_no_salt.final_deficit_effect:,.0f}B")
print(f"Additional cost from removing SALT cap: ${result_no_salt.final_deficit_effect - result_tcja.final_deficit_effect:,.0f}B")

## 3. Corporate Tax Policies

Score corporate rate changes with pass-through effects and international provisions.

In [ ]:
from fiscal_model.corporate import CorporateTaxPolicy, create_biden_corporate_rate_only

# Biden proposal: 21% -> 28%
biden_corp = create_biden_corporate_rate_only()
result_corp = scorer.score_policy(biden_corp, dynamic=False)

print("Biden Corporate Rate Increase (21% -> 28%)")
print(f"10-Year Revenue: ${-result_corp.final_deficit_effect:,.0f}B")
print(f"CBO Official: $1,347B")
print(f"Error: {abs(-result_corp.final_deficit_effect - 1347) / 1347 * 100:.1f}%")

In [ ]:
# Custom corporate policy with international provisions
custom_corp = CorporateTaxPolicy(
    name="Corporate Reform Package",
    rate_change=0.04,  # +4pp (21% -> 25%)
    corporate_elasticity=0.25,
    include_passthrough_effects=True,
    gilti_rate_change=0.05,  # Increase GILTI rate
    eliminate_fdii=False,
)
result_custom = scorer.score_policy(custom_corp, dynamic=False)

print(f"Custom Corporate Reform: ${-result_custom.final_deficit_effect:,.0f}B revenue")

## 4. Tax Credits (CTC, EITC)

Score expansions to refundable tax credits with phase-in/out modeling.

In [ ]:
from fiscal_model.credits import TaxCreditPolicy

# Biden-style CTC expansion (similar to 2021 American Rescue Plan)
ctc_expansion = TaxCreditPolicy(
    name="Biden CTC Expansion",
    credit_type="ctc",
    credit_amount_change=1600,  # Increase per-child credit
    eligible_population=74_000_000,  # Children eligible
    phase_in_rate=0.0,
    phase_out_start=150000,
    phase_out_rate=0.05,
    refundable=True,
    duration_years=10
)
result_ctc = scorer.score_policy(ctc_expansion, dynamic=False)

print("Biden CTC Expansion")
print(f"10-Year Cost: ${result_ctc.final_deficit_effect:,.0f}B")
print(f"CBO/JCT estimate: ~$1,600B")

In [ ]:
# EITC expansion for childless workers
eitc_expansion = TaxCreditPolicy(
    name="EITC Expansion - Childless",
    credit_type="eitc",
    credit_amount_change=1500,
    eligible_population=20_000_000,
    phase_in_rate=0.153,
    phase_out_start=11000,
    phase_out_rate=0.0765,
    refundable=True,
    duration_years=10
)
result_eitc = scorer.score_policy(eitc_expansion, dynamic=False)

print(f"EITC Expansion: ${result_eitc.final_deficit_effect:,.0f}B cost")

## 5. Payroll Tax Policies

Score Social Security cap reforms and Medicare/NIIT expansions.

In [ ]:
from fiscal_model.payroll import PayrollTaxPolicy

# Social Security "Donut Hole" - apply SS tax above $250K
ss_donut = PayrollTaxPolicy(
    name="SS Donut Hole ($250K)",
    payroll_type="ss_donut",
    threshold=250_000,
    rate_change=0.124,  # Full 12.4% employer+employee
    duration_years=10
)
result_ss = scorer.score_policy(ss_donut, dynamic=False)

print("Social Security Donut Hole ($250K threshold)")
print(f"10-Year Revenue: ${-result_ss.final_deficit_effect:,.0f}B")
print(f"CBO estimate: ~$2,700B")

In [ ]:
# NIIT expansion to cover more income types
niit_expansion = PayrollTaxPolicy(
    name="NIIT Expansion",
    payroll_type="niit",
    threshold=400_000,
    rate_change=0.038,  # 3.8% NIIT rate
    expand_niit_base=True,  # Apply to pass-through income
    duration_years=10
)
result_niit = scorer.score_policy(niit_expansion, dynamic=False)

print(f"NIIT Expansion: ${-result_niit.final_deficit_effect:,.0f}B revenue")

## 6. Estate Tax

Score changes to estate tax exemption and rates.

In [ ]:
from fiscal_model.estate import EstateTaxPolicy

# Biden estate reform (lower exemption, higher rate)
estate_biden = EstateTaxPolicy(
    name="Biden Estate Reform",
    exemption_change=-9_500_000,  # Reduce from ~$13M to ~$3.5M
    rate_change=0.05,  # Increase top rate
    duration_years=10
)
result_estate = scorer.score_policy(estate_biden, dynamic=False)

print("Biden Estate Tax Reform")
print(f"10-Year Revenue: ${-result_estate.final_deficit_effect:,.0f}B")
print(f"CBO estimate: ~$450B")

## 7. Alternative Minimum Tax (AMT)

Score changes to individual and corporate AMT.

In [ ]:
from fiscal_model.amt import AMTPolicy

# Repeal Corporate AMT (CAMT)
repeal_camt = AMTPolicy(
    name="Repeal Corporate AMT",
    amt_type="corporate",
    repeal=True,
    duration_years=10
)
result_camt = scorer.score_policy(repeal_camt, dynamic=False)

print("Repeal Corporate AMT (CAMT)")
print(f"10-Year Cost: ${result_camt.final_deficit_effect:,.0f}B")
print(f"CBO estimate: $220B")

## 8. Tax Expenditures (SALT, Mortgage)

Score changes to major tax expenditures.

In [ ]:
from fiscal_model.tax_expenditures import TaxExpenditurePolicy

# Cap employer health exclusion
cap_health = TaxExpenditurePolicy(
    name="Cap Employer Health Exclusion",
    expenditure_type="employer_health",
    cap_amount=15_000,  # Cap at $15K/year
    duration_years=10
)
result_health = scorer.score_policy(cap_health, dynamic=False)

print("Cap Employer Health Exclusion")
print(f"10-Year Revenue: ${-result_health.final_deficit_effect:,.0f}B")
print(f"CBO estimate: ~$450B")

## 9. Distributional Analysis

Analyze how policies affect different income groups using TPC/JCT methodology.

In [ ]:
from fiscal_model.distribution import DistributionalEngine, IncomeGroupType

# Initialize engine
dist_engine = DistributionalEngine()

# Analyze TCJA extension by quintile
tcja_policy = create_tcja_extension(extend_all=True)
dist_result = dist_engine.analyze_policy(tcja_policy, group_type=IncomeGroupType.QUINTILE)

print("TCJA Extension - Distributional Impact by Quintile")
print(dist_result.to_dataframe().to_string())

In [ ]:
# Analyze by dollar brackets (more detail)
dist_dollars = dist_engine.analyze_policy(tcja_policy, group_type=IncomeGroupType.DOLLAR_BRACKETS)

print("\nTCJA Extension - By Income Bracket")
print(dist_dollars.to_dataframe().to_string())

In [ ]:
# Corporate tax distributional impact (capital/labor incidence)
corp_dist = dist_engine.analyze_policy(biden_corp, group_type=IncomeGroupType.QUINTILE)

print("\nBiden Corporate Rate Increase - Distributional Impact")
print("(75% capital / 25% labor incidence)")
print(corp_dist.to_dataframe().to_string())

## 10. Dynamic Scoring

Model macroeconomic feedback using FRB/US-calibrated multipliers.

In [ ]:
from fiscal_model.models.macro_adapter import FRBUSAdapterLite, MacroScenario

# Initialize FRB/US-calibrated adapter
macro = FRBUSAdapterLite()

# Create scenario from TCJA extension
annual_cost = result_tcja.final_deficit_effect / 10  # ~$460B/year
scenario = MacroScenario(
    name="TCJA Extension",
    description="Full extension of TCJA provisions",
    receipts_change=np.array([annual_cost] * 10),  # Revenue loss each year
)

# Run dynamic simulation
dynamic_result = macro.run(scenario)

print("TCJA Extension - Dynamic Effects")
print(f"Cumulative GDP Effect: {dynamic_result.cumulative_gdp_effect:.2f}%-years")
print(f"Average Annual GDP: {np.mean(dynamic_result.gdp_level) * 100:.2f}%")
print(f"Revenue Feedback: ${dynamic_result.cumulative_revenue_feedback:,.0f}B")
print(f"Employment Effect: {np.mean(dynamic_result.employment) * 100:.2f}% average")

In [ ]:
# Plot dynamic effects over time
import matplotlib.pyplot as plt

years = list(range(1, 11))

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# GDP effect
axes[0, 0].plot(years, [g * 100 for g in dynamic_result.gdp_level], 'b-', linewidth=2)
axes[0, 0].set_title('GDP Effect')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('% Change')
axes[0, 0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Employment effect
axes[0, 1].plot(years, [e * 100 for e in dynamic_result.employment], 'g-', linewidth=2)
axes[0, 1].set_title('Employment Effect')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('% Change')
axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Revenue feedback
axes[1, 0].bar(years, dynamic_result.revenue_feedback, color='orange', alpha=0.7)
axes[1, 0].set_title('Revenue Feedback')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('$B')

# Interest rate effect
axes[1, 1].plot(years, [r * 100 for r in dynamic_result.interest_rate], 'r-', linewidth=2)
axes[1, 1].set_title('Interest Rate Effect (Crowding Out)')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('pp Change')

plt.suptitle('TCJA Extension - Dynamic Scoring Results', fontsize=14)
plt.tight_layout()
plt.show()

## 11. Policy Packages

Combine multiple policies into packages for comprehensive analysis.

In [ ]:
# Define a "Progressive Revenue" package
progressive_package = [
    ("High-Income Tax Increase", TaxPolicy(
        name="Income Tax +2.6pp on $400K+",
        rate_change=0.026,
        affected_income_threshold=400_000,
        duration_years=10
    )),
    ("Corporate Rate Increase", create_biden_corporate_rate_only()),
    ("Estate Reform", EstateTaxPolicy(
        name="Estate Reform",
        exemption_change=-9_500_000,
        rate_change=0.05,
        duration_years=10
    )),
]

# Score each policy
print("Progressive Revenue Package")
print("=" * 50)
total_revenue = 0
for name, policy in progressive_package:
    result = scorer.score_policy(policy, dynamic=False)
    revenue = -result.final_deficit_effect
    total_revenue += revenue
    print(f"{name}: ${revenue:,.0f}B")

print("=" * 50)
print(f"PACKAGE TOTAL: ${total_revenue:,.0f}B revenue")

In [ ]:
# Create summary DataFrame
package_data = []
for name, policy in progressive_package:
    result = scorer.score_policy(policy, dynamic=False)
    package_data.append({
        'Policy': name,
        'Static Effect ($B)': result.static_revenue_effect,
        'Behavioral Offset ($B)': result.behavioral_offset,
        'Net 10-Year ($B)': result.final_deficit_effect
    })

df_package = pd.DataFrame(package_data)
print(df_package.to_string(index=False))

## 12. Accessing IRS SOI Data

The model uses real IRS Statistics of Income data for taxpayer counts and income distributions.

In [ ]:
from fiscal_model.data.irs_soi import IRSSOIData

# Initialize IRS data loader
irs = IRSSOIData()

# Get filers above $400K threshold
bracket_info = irs.get_filers_by_bracket(year=2022, threshold=400_000)

print("IRS SOI Data - Filers Above $400K (2022)")
for key, value in bracket_info.items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:,.0f}")
    else:
        print(f"  {key}: {value}")

## 13. Capital Gains Policies

Score capital gains rate changes with time-varying elasticity (CBO/JCT methodology).

In [ ]:
# Biden capital gains proposal
cap_gains_policy = CapitalGainsPolicy(
    name="Tax Cap Gains as Ordinary Income ($1M+)",
    rate_change=0.196,  # From 23.8% to ~43.4%
    affected_income_threshold=1_000_000,
    baseline_realizations_billions=1000,
    baseline_capital_gains_rate=0.238,
    short_run_elasticity=0.8,  # High initial response (timing)
    long_run_elasticity=0.4,   # Lower permanent response
    transition_years=3,
    duration_years=10
)
result_cg = scorer.score_policy(cap_gains_policy, dynamic=False)

print("Capital Gains as Ordinary Income ($1M+)")
print(f"10-Year Revenue: ${-result_cg.final_deficit_effect:,.0f}B")
print("Note: Time-varying elasticity reduces revenue in early years due to timing effects")

## 14. Validation Against Official Scores

Compare model estimates to official CBO/JCT scores.

In [ ]:
from fiscal_model.validation.cbo_scores import CBO_OFFICIAL_SCORES

# Show validation results for key policies
validation_policies = [
    ("TCJA Full Extension", tcja_full, 4600),
    ("Biden Corporate 28%", biden_corp, -1347),
    ("Repeal Corporate AMT", repeal_camt, 220),
]

print("Model Validation vs Official CBO/JCT Scores")
print("=" * 60)
print(f"{'Policy':<25} {'Model':>10} {'Official':>10} {'Error':>10}")
print("-" * 60)

for name, policy, official in validation_policies:
    result = scorer.score_policy(policy, dynamic=False)
    model_score = result.final_deficit_effect
    error = abs(model_score - official) / abs(official) * 100
    print(f"{name:<25} ${model_score:>8,.0f}B ${official:>8,}B {error:>9.1f}%")

## Summary

This notebook demonstrated:

1. **Basic scoring** - Tax policies with rate changes and thresholds
2. **TCJA extension** - Component-based scoring matching CBO estimates
3. **Corporate tax** - Rate changes with pass-through and international effects
4. **Tax credits** - CTC/EITC with phase-in/out modeling
5. **Payroll tax** - Social Security reforms, NIIT expansion
6. **Estate tax** - Exemption and rate changes
7. **AMT** - Individual and corporate AMT
8. **Tax expenditures** - SALT, mortgage, employer health
9. **Distributional analysis** - TPC/JCT-style tables by income group
10. **Dynamic scoring** - FRB/US-calibrated GDP feedback
11. **Policy packages** - Combining multiple policies
12. **IRS SOI data** - Real taxpayer data
13. **Capital gains** - Time-varying elasticity
14. **Validation** - Comparison to official scores

For more details, see:
- `docs/METHODOLOGY.md` - Full methodology documentation
- `docs/ARCHITECTURE.md` - System architecture
- `planning/ROADMAP.md` - Development roadmap